In [44]:
'''
Parse fasta file info, combine shortName, long name, fasta file name
Parse gff3 file info, combine shortName, long name, fasta file name

Aug. 3 added & update
Parse each folder and make table: shortName, LongName, fastaFilename, gffName, gff3Name
consolidate full set of infomation with NA for entry that were not presented 
'''
import pandas as pd
import re
import os
dataPath = "/home/richard/research/1_DataSmall/2_completeFileInfo/"
dataPathLarge = "/home/richard/1_DataLarge/fasta_gff_gtf_07302015/"

In [43]:
#make fastafile ---- genome name pair info
with open(dataPath+"0_genomeName_fileName.txt") as f: 
    fastaInfo = [line.lstrip().rstrip() for line in f if len(line) > 1]

# get genome names for each line of info
fastaGenomeNamePattern = '(.*): Project'
fastaGenomeName = []
for line in fastaInfo:
    matchObj = re.match(fastaGenomeNamePattern, line)
    fastaGenomeName.append(matchObj.group(1))
# get genome file names for each line of info
fastaFileName = []
fastaFileNamePattern = ".*: Project.*\,\s*?(\S*\.gz)"
for line in fastaInfo:
    try:
        matchObj = re.match(fastaFileNamePattern, line)
        fastaFileName.append(matchObj.group(1))
    except:
#         if "unmasked" in line:
        print line

assert len(fastaGenomeName) == len(fastaFileName)
# len(fastaFileName)
# for d in zip(fastaFileName,fastaGenomeName): print d
fastaDict = {x:y for x,y in zip(fastaFileName,fastaGenomeName)}
len(fastaDict)

529

In [13]:
## There are four kinds of annotation files, gtf(very few), gff, gff3(all protein), gff3(filtered model)
## (we only need gff3(filtered protein) annotation for intron comparison) outdated


#make gff3(filtered protein) filename ---- genome name pair info
with open(dataPath+"1_annotationName_fileName.txt") as f: 
    annotationInfo = [line.lstrip().rstrip() for line in f if len(line) > 1]

# get genome names for each line of info
annotationGenomeNamePattern = '(.*): Project'
annotationGenomeName = []
for line in annotationInfo:
    matchObj = re.match(annotationGenomeNamePattern, line)
    annotationGenomeName.append(matchObj.group(1))

# get gff3(filtered protein) names for each line of info
gff3FilteredNamePattern = '.*\,\s*(.*filtered_proteins.*?gff3.gz)\s'
gff3FilteredName = []

for line in annotationInfo:
    try:
        matchObj = re.match(gff3FilteredNamePattern, line)    
        gff3FilteredName.append(matchObj.group(1))
    except:
        gff3FilteredName.append(None)

assert len(annotationGenomeName) == len(gff3FilteredName)
# for d in zip(gff3FilteredName,annotationGenomeName): 
#     if None not in d:
#         print d
# len([item  for item in gff3FilteredName if item !=None])
annotationDict = {x:y for x,y in zip(gff3FilteredName,annotationGenomeName) if x != None and y != None}
len(annotationDict)

382

In [14]:
## check gff3 files that were left off due two in consistent nameing
for line in annotationInfo:
    if ('filtered_proteins' not in line and 
       'all_proteins' not in line and 
       'gff.gz' not in line and
       'GeneCatalog' not in line and 
       'gtf.gz' not in line):
        print line
        print
    
    
## get into each folder(folder name is defined to be the short name)
## capture the folder that have both gff3 filtered protein file and fasta file, then attach long name on it.

def listContain(alist, target):
    '''
    check to see if keyword: target is in any of items in alist
    '''
    return sum(map(lambda x: target in x,alist)) == 1

def getFileName(alist, target):
    for filename in alist:
        if target.lower() in filename.lower():
            return filename
    return "NA"

shortNames = os.listdir(dataPathLarge)

folderNameFiltered = []
completeInfo = [] # shortName, longName, fasta filename, gff3 filtered protein name
for shortName in shortNames:
    filenames = os.listdir(dataPathLarge+"{}".format(shortName))
    if listContain(filenames, "fasta") and listContain(filenames, "filtered_proteins"):
        folderNameFiltered.append(shortName)
        fasta, gff3 = getFileName(filenames, "fasta"), getFileName(filenames, "filtered_proteins")
        assert fastaDict[fasta] == annotationDict[gff3]
        completeInfo.append((shortName, fastaDict[fasta], fasta, gff3))
print len(completeInfo)
completeInfoDF = pd.DataFrame(completeInfo, columns= ['shortName', 'longName', 'fastaName','gff3Name'])
completeInfoDF.sort(columns = 'shortName').to_csv(dataPath+"2_completeFileInfo.csv", index= False) #sorted

Armillaria mellea: Project: 0, Armme1_1.gff3.gz   2 MB;  Wed Nov 19 18:50:20 PST 2014  File information

Dacryopinax sp. DJM 731 SSP1 v1.0: Project: 403529, Dacsp1_GeneModels_FilteredModels1.gff3   8 MB;  Wed Mar 07 16:35:43 PST 2012  File information   md5: 1ab96cb8a322de98df6489f3331e5035

Zymoseptoria ardabiliae STIR04_1.1.1: Project: 0, Zymar1_ExternalModels_genes_20141022.gff3.gz   841 KB;  Thu Oct 23 12:58:06 PDT 2014  File information

Zymoseptoria pseudotritici STIR04_2.2.1: Project: 0, Zymps1_ExternalModels_genes_20141022.gff3.gz   859 KB;  Thu Oct 23 12:59:13 PDT 2014  File information

363


In [23]:
# Aug. 3 updated
## include gff file names, gffs are the most abundant type of annotation
#make gff(GeneCatalog) filename ---- genome name pair info
with open(dataPath+"1_annotationName_fileName.txt") as f: 
    annotationInfo = [line.lstrip().rstrip() for line in f if len(line) > 1]

# get genome names for each line of info
annotationGenomeNamePattern = '(.*): Project'
annotationGenomeName = []
for line in annotationInfo:
    matchObj = re.match(annotationGenomeNamePattern, line)
    annotationGenomeName.append(matchObj.group(1))

# get gff(GeneCatalog) filename for each line of info
gffFilteredNamePattern = '.*\,\s*(.*GeneCatalog.*?gff.gz)\s'
gffFilteredName = []

for line in annotationInfo:
    try:
        matchObj = re.match(gffFilteredNamePattern, line, re.IGNORECASE)    
        gffFilteredName.append(matchObj.group(1))
    except:
        gffFilteredName.append(None)

assert len(annotationGenomeName) == len(gffFilteredName)
# for d in zip(gff3FilteredName,annotationGenomeName): 
#     if None not in d:
#         print d
# len([item  for item in gff3FilteredName if item !=None])
annotationDict = {x:y for x,y in zip(gffFilteredName,annotationGenomeName) if x != None and y != None}
len(annotationDict)
# for item in annotationDict.items():
#     print item

490

In [46]:
## data we have: shortNames(all the name index we have)

completeInfo = [] # shortName, LongName, fastaName, gffName, gff3Name
fastaName = []
gffName = []
gff3Name = []
for shortName in shortNames:
    filenames = os.listdir(dataPathLarge+"{}".format(shortName))
    fasta = getFileName(filenames, "fasta")
    gff3 = getFileName(filenames, "filtered_proteins")
    gff = getFileName(filenames, "GeneCatalog")
#     assert fastaDict[fasta] == annotationDict[gff3]
    try:
        longName = fastaDict[fasta]
        completeInfo.append((shortName, fastaDict[fasta], fasta, gff, gff3))
    except:
        print "No fasta file found", shortName
        completeInfo.append((shortName, "NA", fasta, gff3))

completeTable = pd.DataFrame(completeInfo, columns='shortName, LongName, fastaName, gffName, gff3Name'.split(", "))
completeTable = completeTable.sort(columns = 'shortName')
completeTable.to_csv(dataPath+"fullTableInfo.csv", index= False) #sorted

No fasta file found Sporo1
No fasta file found Neute_matA2
No fasta file found Pospl1
No fasta file found Ustma1
No fasta file found Crypa2
